### [1] 구글 크롤링

In [23]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
from selenium import webdriver
import time

options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(options=options, executable_path='C:/chromedriver_win32/chromedriver.exe')
driver.get('https://www.google.com/maps/')
driver.implicitly_wait(3)
searchbox = driver.find_element_by_css_selector("input#searchboxinput")
searchbox.send_keys("복현1동카페")
# 검색버튼 누르기
time.sleep(2)
searchbutton = driver.find_element_by_css_selector("button#searchbox-searchbutton")
searchbutton.click()

# html = driver.page_source
# soup = BeautifulSoup(html, 'html.parser')
driver.find_element_by_css_selector("#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd > div:nth-child(3) > div > a").click()
time.sleep(3)
driver.find_element_by_css_selector("#QA0Szd > div > div > div.w6VYqd > div.bJzME.Hu9e2e.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf > div:nth-child(3) > div > div > button:nth-child(2) > div.LRkQ2 > div.Gpq6kf.fontTitleSmall").click()


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=111.0.5563.147)


### [2] 카카오 크롤링

#### [2-1] 카카오 url 수집

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from selenium import webdriver #pip install selenium
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import re
import time
import selenium
from selenium.webdriver.common.action_chains import ActionChains
selenium.__version__ 
def collect_kakaomap_url(text):
    path = 'C:/chromedriver_win32/chromedriver.exe'
    source_url = "https://map.kakao.com/"
    driver = webdriver.Chrome(path)
    driver.get(source_url)
    # id="search.keyword.query" 태그 선택=> input 태그 
    searchbox=driver.find_element(By.ID,"search.keyword.query") 
    searchbox.send_keys(text) #url 수집을 원하는 값 입력
    # id="search.keyword.submit" 태그 선택=> input 태그 
    searchbutton=driver.find_element(By.ID,"search.keyword.submit") 
    #script를 실행
    #arguments[0].click(); : 첫번째 매개변수값을 클릭
    driver.execute_script("arguments[0].click();", searchbutton) #검색버튼클릭.
    time.sleep(2) #2초동안 대기 =>  검색 결과가 브라우저에 표시됨
    html = driver.page_source # 현재브라우저의 소스(html)
    soup = BeautifulSoup(html, "html.parser") # BeautifulSoup 분석
    # name="a": a태그
    # attrs={"class":"moreview" : a태그중에 class속성이 moreview인 태그
    moreviews = soup.find_all(name="a", attrs={"class":"moreview"}) # 상세보기들
    page_urls = [] 
    for moreview in moreviews:
        # moreview: 한개의 상세보기. a태그
        page_url = moreview.get("href") # a태그의 href 속성의 값
        page_urls.append(page_url) #상세보기의 href 속성값들 목록. 강남역 고기집의 URL 정보 목록
    # driver.close()    
    more_place = driver.find_element_by_xpath('//*[@id="info.search.place.more"]')

    actions = ActionChains(driver )#스크롤내리기
    actions.move_to_element(more_place).perform()
    time.sleep(1)
    more_place = driver.find_element_by_xpath('/html/body/div[10]/div/div/div/span')
    more_place.click()
    try:
        more_place.click()
    except:
        more_place = driver.find_element_by_xpath('/html/body/div[5]/div[2]/div[1]/div[7]/div[5]/a')
        more_place.click()
    page_num = 3
    while True:
        time.sleep(1.5)
        html = driver.page_source # 현재브라우저의 소스(html)
        soup = BeautifulSoup(html, "html.parser") # BeautifulSoup 분석
        # name="a": a태그
        # attrs={"class":"moreview" : a태그중에 class속성이 moreview인 태그
        moreviews = soup.find_all(name="a", attrs={"class":"moreview"}) # 상세보기들
        
        for moreview in moreviews:
            # moreview: 한개의 상세보기. a태그
            page_url = moreview.get("href") # a태그의 href 속성의 값
            page_urls.append(page_url) #상세보기의 href 속성값들 목록. 강남역 고기집의 URL 정보 목록
        try:# 페이지 버튼마다 반복 없을 시 다음 페이지로 선택
            page_button = driver.find_element_by_css_selector('#info\.search\.page\.no{}'.format(page_num))
            page_num += 1
        except Exception:
            page_button = driver.find_element_by_css_selector('#info\.search\.page\.next')
            page_num = 2
            
        try:#버튼 클릭 및 요소가 없을 시 break
            page_button.click()
        except:
            
            break
    return page_urls

In [26]:
page_urls = collect_kakaomap_url('수분크림')

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=111.0.5563.147)


In [27]:
len(page_urls)

500

In [13]:
page_urls

['https://place.map.kakao.com/95713992',
 'https://place.map.kakao.com/1503746075',
 'https://place.map.kakao.com/2011092566',
 'https://place.map.kakao.com/741391811',
 'https://place.map.kakao.com/168079537',
 'https://place.map.kakao.com/1052874675',
 'https://place.map.kakao.com/1648266796',
 'https://place.map.kakao.com/27238067',
 'https://place.map.kakao.com/26573718',
 'https://place.map.kakao.com/263830255',
 'https://place.map.kakao.com/1511737742',
 'https://place.map.kakao.com/1780387311',
 'https://place.map.kakao.com/56929632',
 'https://place.map.kakao.com/27511865',
 'https://place.map.kakao.com/279490770',
 'https://place.map.kakao.com/785573131',
 'https://place.map.kakao.com/1907052666',
 'https://place.map.kakao.com/27463588',
 'https://place.map.kakao.com/12997393',
 'https://place.map.kakao.com/609286803',
 'https://place.map.kakao.com/855757075',
 'https://place.map.kakao.com/7912852',
 'https://place.map.kakao.com/2000332368',
 'https://place.map.kakao.com/13324

---
# URL 1개에 대한 df생성
---

In [35]:
# 상세보기에 조회된 고기집 목록을 조회
path = 'C:/chromedriver_win32/chromedriver.exe'
df = pd.DataFrame() # 컬럼만 조회
driver = webdriver.Chrome(path) # 브라우저 실행
# for page in page_urls:
df = pd.DataFrame()
time.sleep(1)
# for page in page_urls:
driver.get(page_urls[30]) # 식당 데이터 상세화면
time.sleep(2) # 2초 쉬기
html = driver.page_source # html 소스 데이터 (page 소스를 html에 세팅)
soup = BeautifulSoup(html, 'html.parser')


total_strong = soup.find(name="strong", attrs={"class": "total_evaluation"})
try:
    tot_num = total_strong.find(name="span", attrs={"class": "color_b"}) # 등록된 리뷰 갯수
except:
    pass
    # continue
if int(tot_num.text) == 0:
    pass
    # continue
more_text_num= ((int(tot_num.text)-3)//5)
time.sleep(1)
while True:
    if more_text_num >3:
        # try:
        more_text_in_store = driver.find_element_by_css_selector('span.txt_more')
            # more_text_in_store = driver.find_element_by_xpath('//*[@id="mArticle"]/div[7]/div[3]/a/span[1]')
        # except:
        #     more_text_in_store = driver.find_element_by_xpath('//*[@id="mArticle"]/div[8]/div[3]/a/span[1]')
        # else:
        #     more_text_in_store = driver.find_element_by_xpath('//*[@id="mArticle"]/div[6]/div[3]/a/span[1]')
        actions = ActionChains(driver)#스크롤내리기
        actions.move_to_element(more_text_in_store).perform()
        more_text_in_store.click()
        time.sleep(0.5)
    else:
        break
    more_text_num -= 1



    #continue
    # try 구문에서 오류 발생된 경우 다음 페이지로 이동 
    # 리뷰의 1페이지 정보를 df에 저장
rate_list =[]
review_list =[]

html = driver.page_source # html 소스 데이터 (page 소스를 html에 세팅)
soup = BeautifulSoup(html, 'html.parser')
try:        
    store_name = soup.find(name = 'h2', attrs = {'class':'tit_location'})#가게이름
    addr = soup.find(name = 'span', attrs = {'class':'txt_address'})#가게주소
    store_operate_day = soup.find(name = 'span', attrs = {'class':'txt_operation'})#영업일 및 영업시간

    
    img_urls = soup.find(name='span', attrs={'class':'bg_present'})
    img_urls = img_urls.get('style')
    start_idx = img_urls.find('url(') + 4  # "url(" 다음 인덱스부터 시작
    end_idx = img_urls.find(')', start_idx)
    url = img_urls[start_idx:end_idx]
    

    total_strong = soup.find(name="strong", attrs={"class": "total_evaluation"})
    tot_num = total_strong.find(name="span", attrs={"class": "color_b"}) # 등록된 리뷰 갯수
    contents_div = soup.find(name='div', attrs={"class": "evaluation_review"}) #리뷰 영역. 별점, 내용
    reviewer_average_score = contents_div.find_all(name="span", attrs={"class": "txt_desc"})#리뷰어의 평균평점
    #reviewer_num = soup.find(name="span", attrs={"class": "text_desc"})#리뷰어의 평균평점
    rates=contents_div.find_all(name="span", attrs={"class": "ico_star star_rate"}) # 별점값 목록
    reviews = contents_div.find_all(name="p", attrs={"class": "txt_comment"}) # 리뷰 목록
except:
    pass
    #continue
    # try 구문에서 오류 발생된 경우 다음 페이지로 이동 
#     # 리뷰의 1페이지 정보를 df에 저장

rate_list =[]
review_list =[]
reviewer_average_score_list = []
reviewer_num_list = []

# for i in range(reviewer_average_score):
#     reviewer_average_score_list.append(reviewer_average_score)
    
for i in range(len(rates)):
    rate_value = rates[i].find(name='span', attrs={"class": "ico_star"})
    style_value = rate_value.get('style')
    if style_value == 'width:20%;':
        rate = 1
    elif style_value == 'width:40%;':
        rate = 2
    elif style_value == 'width:60%;':
        rate = 3
    elif style_value == 'width:80%;':
        rate = 4
    elif style_value == 'width:100%;':
        rate = 5
    rate_list.append(rate)    
for review in reviews:
    rev_span = review.find(name="span")
    if rev_span is not None and rev_span.text:
        rev = rev_span.text
    else:
        rev = ''
    review_list.append(rev)
for i in range(len(reviewer_average_score)):
    if i%2 == 0:
        reviewer_average_score_list.append(i.text)
    else:
        reviewer_num_list.append(i.text)

store_name = store_name.text
addr = addr.text
addr = [i.strip() for i in addr.split('\n')]
addr = ' '.join(addr)
store_operate_day = store_operate_day.text
store_operate_day = store_operate_day.replace('\n', '')

#이미지 url 추가하기


df['리뷰'] = review_list
df['별점'] = rate_list
df['리뷰자 평균 평점'] = reviewer_average_score_list
df['리뷰자 누적 리뷰 개수'] = reviewer_num_list
df['영업 시간'] = store_operate_day
df['이미지url'] = url
df['가게명'] = store_name
df['가게 주소'] = addr
print(df)


AttributeError: 'int' object has no attribute 'text'

---
# 반복문으로 돌리기
- 리뷰 없으면 continue
---

In [36]:
path = 'C:/chromedriver_win32/chromedriver.exe'
total_df = pd.DataFrame(columns=['리뷰','별점', '리뷰자 평균 평점','리뷰자 누적 리뷰 개수', '영업 시간','이미지url','가게명','가게 주소']) # 컬럼만 조회
driver = webdriver.Chrome(path) # 브라우저 실행
for i in page_urls:
    df = pd.DataFrame()
    time.sleep(1)
    # for page in page_urls:
    driver.get(i) # 식당 데이터 상세화면
    time.sleep(2) # 2초 쉬기
    html = driver.page_source # html 소스 데이터 (page 소스를 html에 세팅)
    soup = BeautifulSoup(html, 'html.parser')
    total_strong = soup.find(name="strong", attrs={"class": "total_evaluation"})
    try:
        tot_num = total_strong.find(name="span", attrs={"class": "color_b"}) # 등록된 리뷰 갯수
    except:
        pass
        # continue
    if int(tot_num.text) == 0:
        pass
        # continue
    more_text_num= ((int(tot_num.text)-3)//5)
    time.sleep(1)
    while True:
        if more_text_num >3:
            # try:
            more_text_in_store = driver.find_element_by_css_selector('span.txt_more')
                # more_text_in_store = driver.find_element_by_xpath('//*[@id="mArticle"]/div[7]/div[3]/a/span[1]')
            # except:
            #     more_text_in_store = driver.find_element_by_xpath('//*[@id="mArticle"]/div[8]/div[3]/a/span[1]')
            # else:
            #     more_text_in_store = driver.find_element_by_xpath('//*[@id="mArticle"]/div[6]/div[3]/a/span[1]')
            actions = ActionChains(driver)#스크롤내리기
            actions.move_to_element(more_text_in_store).perform()
            more_text_in_store.click()
            time.sleep(0.5)
        else:
            break
        more_text_num -= 1



        #continue
        # try 구문에서 오류 발생된 경우 다음 페이지로 이동 
        # 리뷰의 1페이지 정보를 df에 저장


    html = driver.page_source # html 소스 데이터 (page 소스를 html에 세팅)
    soup = BeautifulSoup(html, 'html.parser')
    try:        
        store_name = soup.find(name = 'h2', attrs = {'class':'tit_location'})#가게이름
        addr = soup.find(name = 'span', attrs = {'class':'txt_address'})#가게주소
        store_operate_day = soup.find(name = 'span', attrs = {'class':'txt_operation'})#영업일 및 영업시간

        
        img_urls = soup.find(name='span', attrs={'class':'bg_present'})
        img_urls = img_urls.get('style')
        start_idx = img_urls.find('url(') + 4  # "url(" 다음 인덱스부터 시작
        end_idx = img_urls.find(')', start_idx)
        url = img_urls[start_idx:end_idx]
        

        total_strong = soup.find(name="strong", attrs={"class": "total_evaluation"})
        tot_num = total_strong.find(name="span", attrs={"class": "color_b"}) # 등록된 리뷰 갯수
        contents_div = soup.find(name='div', attrs={"class": "evaluation_review"}) #리뷰 영역. 별점, 내용
        reviewer_average_score = contents_div.find_all(name="span", attrs={"class": "txt_desc"})#리뷰어의 평균평점
        #reviewer_num = soup.find(name="span", attrs={"class": "text_desc"})#리뷰어의 평균평점
        rates=contents_div.find_all(name="span", attrs={"class": "ico_star star_rate"}) # 별점값 목록
        reviews = contents_div.find_all(name="p", attrs={"class": "txt_comment"}) # 리뷰 목록
    except:
        pass
        #continue
        # try 구문에서 오류 발생된 경우 다음 페이지로 이동 
    #     # 리뷰의 1페이지 정보를 df에 저장

    rate_list =[]
    review_list =[]
    reviewer_average_score_list = []
    reviewer_num_list = []


    for i in range(len(rates)):
        rate_value = rates[i].find(name='span', attrs={"class": "ico_star"})
        style_value = rate_value.get('style')
        if style_value == 'width:20%;':
            rate = 1
        elif style_value == 'width:40%;':
            rate = 2
        elif style_value == 'width:60%;':
            rate = 3
        elif style_value == 'width:80%;':
            rate = 4
        elif style_value == 'width:100%;':
            rate = 5
        rate_list.append(rate)    
    for review in reviews:
        rev_span = review.find(name="span")
        if rev_span is not None and rev_span.text:
            rev = rev_span.text
        else:
            rev = ''
        review_list.append(rev)
    for i in range(len(reviewer_average_score)):
        if i%2 == 0:
            reviewer_average_score_list.append(reviewer_average_score[i].text)
        else:
            reviewer_num_list.append(reviewer_average_score[i].text)

    store_name = store_name.text
    addr = addr.text
    addr = [i.strip() for i in addr.split('\n')]
    addr = ' '.join(addr)
    store_operate_day = store_operate_day.text
    store_operate_day = store_operate_day.replace('\n', '')

    #이미지 url 추가하기

    df['리뷰'] = review_list
    df['별점'] = rate_list
    df['리뷰자 평균 평점'] = reviewer_average_score_list
    df['리뷰자 누적 리뷰 개수'] = reviewer_num_list
    df['영업 시간'] = store_operate_day
    df['이미지url'] = url
    df['가게명'] = store_name
    df['가게 주소'] = addr
    print(df)
    total_df = pd.concat([total_df,df])


                                                    리뷰  별점 리뷰자 평균 평점  \
0    칼국수는 별로였지만 김치가 존맛이어서 김치 먹으려규 칼국수 먹었음 김치에 마늘향이 ...   4        10   
1                                          바퀴벌레 국수 맛존탱   1        11   
2                                                        5        27   
3                                                        5        38   
4    시간지나면 생각나는 맛.부들부들한 면.. 맛=평타침자리는 많지만 테이블 협소, 웨이...   4       138   
..                                                 ...  ..       ...   
933  언제나맛있는곳. 꼬들꼬들한 밥도 칼국수를남기더라도 한입먹어야 함. 김치도맛나구ㅠ.ㅠ...   4        85   
934                    명동교자보단 베테랑 칼국수가 내스타일인듯. 만두는 맛있음   3        88   
935  별점 기준1) 재방문 의사 없음.2) 찾아서 갈만큼은 아님. 주로 가성비 떨어짐.3...   4       163   
936                        칼국수, 만두는 맛있으나비빔국수는 그저 그런 정도   3       195   
937  오빠랑 배고파서 가자마자 주문이후 10분만에 싹쓸이진짜 맛있어!!!!♥︎ Love ...   5     1,464   

    리뷰자 누적 리뷰 개수             영업 시간  \
0            3.6  매일 10:30 ~ 21:30   
1            4.2  매일 10:30 ~ 21:30   
2            5.0  매일 

In [ ]:
total_df

In [10]:
total_df.to_csv('sample.csv')

In [97]:
html = driver.page_source # html 소스 데이터 (page 소스를 html에 세팅)
soup = BeautifulSoup(html, 'html.parser')
try:
    store_name = soup.find(name = 'h2', attrs = {'class':'tit_location'})
    addr = soup.find(name = 'span', attrs = {'class':'txt_address'})
    total_strong = soup.find(name="strong", attrs={"class": "total_evaluation"})
    tot_num = total_strong.find(name="span", attrs={"class": "color_b"}) # 등록된 리뷰 갯수
    contents_div = soup.find(name='div', attrs={"class": "evaluation_review"}) #리뷰 영역. 별점, 내용
    rates=contents_div.find_all(name="span", attrs={"class": "ico_star star_rate"}) # 별점값 목록
    reviews = contents_div.find_all(name="p", attrs={"class": "txt_comment"}) # 리뷰 목록
except:
    pass
    #continue
     # try 구문에서 오류 발생된 경우 다음 페이지로 이동 
#     # 리뷰의 1페이지 정보를 df에 저장
rate_list =[]
review_list =[]
for i in range(len(rates)):
    rate_value = rates[i].find(name='span', attrs={"class": "ico_star"})
    style_value = rate_value.get('style')
    if style_value == 'width:20%;':
        rate = 1
    elif style_value == 'width:40%;':
        rate = 2
    elif style_value == 'width:60%;':
        rate = 3
    elif style_value == 'width:80%;':
        rate = 4
    elif style_value == 'width:100%;':
        rate = 5
    rate_list.append(rate)    
for review in reviews:
    rev_span = review.find(name="span")
    if rev_span is not None and rev_span.text:
        rev = rev_span.text
    else:
        rev = ''
    review_list.append(rev)
store_name = store_name.text
addr = addr.text 
df['score'] = rate_list
df['review'] = review_list
df['store_name'] = store_name

ValueError: Length of values (228) does not match length of index (3)

In [95]:
df

,score,review,store_name
0,5,치즈닭갈비 최고..! 치즈 풍미가 장난이 아니에요조리되어 나와서 바로 먹을 수 있고...,장인닭갈비 강남점
1,5,치즈 있는 닭갈비 처음 먹어봤는데 진짜 맛도리였어요. 여기 닭갈비가 매워서 치즈는 ...,장인닭갈비 강남점
2,3,맛은 그냥 보통임. 가운데에 치즈가 있어서 보기에는 괜춘함.,장인닭갈비 강남점


In [83]:
html = driver.page_source # html 소스 데이터 (page 소스를 html에 세팅)
soup = BeautifulSoup(html, 'html.parser')

contents_div = soup.find(name='div', attrs={"class": "evaluation_review"}) #리뷰 영역. 별점, 내용
rates=contents_div.find_all(name="span", attrs={"class": "ico_star star_rate"}) # 별점값 목록
reviews = contents_div.find_all(name="p", attrs={"class": "txt_comment"}) # 리뷰 목록

In [84]:
reviews

[<p class="txt_comment"><span>치즈닭갈비 최고..! 치즈 풍미가 장난이 아니에요<br/>조리되어 나와서 바로 먹을 수 있고 양이 많아서 좋았어요<br/>배불러서 볶음밥 못먹음 ㅜㅜ</span><button class="btn_fold" type="button">더보기</button></p>,
 <p class="txt_comment"><span>치즈 있는 닭갈비 처음 먹어봤는데 진짜 맛도리였어요. 여기 닭갈비가 매워서 치즈는 필수.. 그리고 강남역 한복판.. 사람들로 엄청 북적이는데 가게 내부는 넓어서 괜찮았어요. 저녁에는 시간에 따라 웨이팅도 있는듯.</span><button class="btn_fold" type="button">더보기</button></p>,
 <p class="txt_comment"><span>맛은 그냥 보통임. 가운데에 치즈가 있어서 보기에는 괜춘함.</span><button class="btn_fold" type="button">더보기</button></p>,
 <p class="txt_comment txt_fold"><span>3/3일 방문 저녁 9시쯤 방문하여 밥먹는데 벽에서 쥐나와서 활보함... 쥐나왔는데 직원들은 별 개의치 않고 손님들만 개 기겁함 살다살다 식당에서 큰 쥐가 돌아다니는거 처음보네 덕분에 집갈 때 그림자만 봐도 쥐인가 식겁함... 어지간하면 댓글 안남기는데 이게 첫 댓글임</span><button class="btn_fold" type="button">더보기</button></p>,
 <p class="txt_comment"><span>직원분이 너~무 불친절하세요<br/>불키라는 말도 안해주고 계속 명령조로 얘기하십니다ㅎㅎ</span><button class="btn_fold" type="button">더보기</button></p>,
 <p class="txt_comment"><span>닭갈비 맛있습니다. 늘 사람이 많고 정신 없어서 서비스가 좀 아쉽습니다</span><button clas

In [ ]:
more_text_in_store = driver.find_element_by_xpath('//*[@id="mArticle"]/div[7]/div[3]/a/span[1]')
actions = ActionChains(driver)#스크롤내리기
actions.move_to_element(more_text_in_store).perform()

In [74]:
# while True:

    # 페이지 숫자의 테그
text_more = driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > a > span.txt_more')
actions = ActionChains(driver) #스크롤내리기
actions.move_to_element(text_more).perform()
text_more.click()
time.sleep(2)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
contents_div = soup.find(name='div', attrs={"class": "evaluation_review"}) #리뷰 영역. 별점, 내용
rates=contents_div.find_all(name="span", attrs={"class": "ico_star star_rate"}) # 별점값 목록
reviews = contents_div.find_all(name="p", attrs={"class": "txt_comment"}) # 리뷰 목록
review_list =[]
for i in range(len(rates)):
    rate_value = rates[i].find(name='span', attrs={"class": "ico_star"})
    style_value = rate_value.get('style')
    if style_value == 'width:20%;':
        rate = 1
    elif style_value == 'width:40%;':
        rate = 2
    elif style_value == 'width:60%;':
        rate = 3
    elif style_value == 'width:80%;':
        rate = 4
    elif style_value == 'width:100%;':
        rate = 5
    rate_list.append(rate)
for review in reviews:
    rev_span = review.find(name="span")
    if rev_span is not None and rev_span.text:
        rev = rev_span.text
    else:
        rev = ''
    review_list.append(rev)
    more_df = pd.DataFrame(rates, review_list, store_name, addr, columns= ['score', 'review', 'store_name', 'addr'])   
#         for rate, review in zip(rates, reviews):
#             row=[rate.text[0], review.find(name="span").text]
#             series = pd.Series(row, index=df.columns)
#             df = df.append(series, ignore_index=True)
    # except:
    #     break

SyntaxError: invalid syntax (2358820242.py, line 41)

In [66]:
df

,score,review,store_name
0,5,치즈닭갈비 최고..! 치즈 풍미가 장난이 아니에요조리되어 나와서 바로 먹을 수 있고...,장인닭갈비 강남점
1,5,치즈 있는 닭갈비 처음 먹어봤는데 진짜 맛도리였어요. 여기 닭갈비가 매워서 치즈는 ...,장인닭갈비 강남점
2,3,맛은 그냥 보통임. 가운데에 치즈가 있어서 보기에는 괜춘함.,장인닭갈비 강남점


In [73]:
more_df

NameError: name 'more_df' is not defined